In [ ]:
import os
import json
import logging
import re
from typing import List, Dict, Any, Optional
from pathlib import Path
from datetime import datetime

# Core imports
import autogen
from autogen import ConversableAgent, GroupChat, GroupChatManager
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# LangChain imports
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document, HumanMessage

# ChromaDB imports
import chromadb
from chromadb.config import Settings

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)


In [ ]:
# Azure Configuration
AZURE_CONFIG = {
    "api_key": ,
    "base_url": "",
    "api_version": "2025-01-01-preview",
    "embedding_deployment": "text-embedding-ada-002",
    "gpt_deployment": "gpt-4o"
}

In [ ]:



class InsuranceRAGSystem:
    """Enhanced Insurance RAG system with ChromaDB backend for document retrieval."""
    
    def __init__(self, data_folder: str = "data", persist_directory: str = "chroma_db"):
        """Initialize the RAG system with data folder and ChromaDB settings."""
        self.data_folder = data_folder
        self.persist_directory = persist_directory
        self.collection_name = "insurance_documents"
        self.embeddings = None
        self.vectorstore = None
        self.llm = None
        self.chroma_client = None
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        self.setup_azure_clients()
        self.setup_chromadb()
        
    def setup_azure_clients(self) -> None:
        """Initialize Azure OpenAI clients for embeddings and LLM."""
        try:
            self.embeddings = AzureOpenAIEmbeddings(
                azure_deployment=AZURE_CONFIG["embedding_deployment"],
                openai_api_version=AZURE_CONFIG["api_version"],
                azure_endpoint=AZURE_CONFIG["base_url"],
                openai_api_key=AZURE_CONFIG["api_key"]
            )
            self.llm = AzureChatOpenAI(
                azure_deployment=AZURE_CONFIG["gpt_deployment"],
                openai_api_version=AZURE_CONFIG["api_version"],
                azure_endpoint=AZURE_CONFIG["base_url"],
                openai_api_key=AZURE_CONFIG["api_key"],
                temperature=0.1,
                max_retries=3,
                timeout=30
            )
            logger.info("Azure OpenAI clients initialized successfully")
        except Exception as e:
            logger.error(f"Failed to initialize Azure clients: {e}")
            raise
    
    def setup_chromadb(self) -> None:
        """Initialize ChromaDB client for persistent storage."""
        try:
            os.makedirs(self.persist_directory, exist_ok=True)
            self.chroma_client = chromadb.PersistentClient(
                path=self.persist_directory,
                settings=Settings(anonymized_telemetry=False, allow_reset=True)
            )
            logger.info(f"ChromaDB client initialized at: {self.persist_directory}")
        except Exception as e:
            logger.error(f"Failed to initialize ChromaDB: {e}")
            raise
    
    def load_and_process_documents(self) -> List[Document]:
        """Load and process insurance documents from various formats."""
        try:
            documents = []
            for ext, loader_class in [(".pdf", PyPDFLoader), (".txt", TextLoader), (".docx", Docx2txtLoader)]:
                files = list(Path(self.data_folder).glob(f"**/*{ext}"))
                for file in files:
                    try:
                        loader = loader_class(str(file), encoding='utf-8' if ext == ".txt" else None)
                        docs = loader.load_and_split()
                        documents.extend(docs)
                        logger.info(f"Loaded {ext[1:].upper()}: {file.name} ({len(docs)} pages)")
                    except Exception as e:
                        logger.warning(f"Error loading {file}: {e}")
            
            if not documents:
                logger.info("No documents found. Creating sample...")
                sample_doc = Document(
                    page_content="""Comprehensive Insurance Policy Guide:
                    AUTO INSURANCE:
                    - Liability Coverage: Bodily injury $100k/person, $300k/accident
                    - Collision: Deductible $500, covers at-fault accidents
                    - Comprehensive: Covers theft, weather, vandalism
                    - PIP: Medical expenses coverage $10k/person
                    HOME INSURANCE:
                    - Dwelling Coverage: Replacement cost up to $500k
                    - Personal Property: Actual cash value coverage
                    - Liability: $300k coverage for property damage
                    - ALE: Additional living expenses up to $100k
                    LIFE INSURANCE:
                    - Term Life: 20-year term, $500k coverage, $50/month
                    - Whole Life: Permanent coverage, cash value accumulation
                    - Universal Life: Flexible premiums, adjustable benefits
                    SPECIAL RIDERS:
                    - Critical Illness Rider: $50k lump sum payment
                    - Accidental Death: 2x benefit for accidental death
                    - Waiver of Premium: Waives premiums if disabled""",
                    metadata={"source": "sample_policy.txt", "type": "sample"}
                )
                documents = [sample_doc]
            
            if documents:
                text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=1500, chunk_overlap=300, length_function=len,
                    separators=["\n\n•", "\n•", "\n\n", "\n", " ", ""]
                )
                split_docs = text_splitter.split_documents(documents)
                logger.info(f"Split {len(documents)} docs into {len(split_docs)} chunks")
                return split_docs
            return []
        except Exception as e:
            logger.error(f"Document processing error: {e}")
            return []
    
    def create_or_load_vectorstore(self, documents: Optional[List[Document]] = None, force_recreate: bool = False) -> None:
        """Create or load ChromaDB vectorstore with documents."""
        try:
            existing_collections = [col.name for col in self.chroma_client.list_collections()]
            collection_exists = self.collection_name in existing_collections
            if collection_exists and not force_recreate:
                logger.info(f"Loading existing collection: {self.collection_name}")
                self.vectorstore = Chroma(
                    client=self.chroma_client,
                    collection_name=self.collection_name,
                    embedding_function=self.embeddings
                )
                collection = self.chroma_client.get_collection(self.collection_name)
                logger.info(f"Loaded {collection.count()} documents")
            else:
                if collection_exists and force_recreate:
                    logger.info(f"Recreating collection: {self.collection_name}")
                    self.chroma_client.delete_collection(self.collection_name)
                else:
                    logger.info(f"Creating new collection: {self.collection_name}")
                if not documents:
                    documents = self.load_and_process_documents()
                if not documents:
                    raise ValueError("No documents available")
                self.vectorstore = Chroma.from_documents(
                    documents=documents, embedding=self.embeddings,
                    client=self.chroma_client, collection_name=self.collection_name,
                    persist_directory=self.persist_directory
                )
                logger.info(f"Created vectorstore with {len(documents)} chunks")
                self.vectorstore.persist()
        except Exception as e:
            logger.error(f"Vectorstore error: {e}")
            raise
    
    def retrieve_context(self, query: str, k: int = 4) -> str:
        """Retrieve relevant context for a query from ChromaDB."""
        try:
            if not self.vectorstore:
                return "No knowledge base available"
            results = self.vectorstore.similarity_search_with_score(query, k=k)
            context_parts = [f"[Relevance: {1 - score:.2f}] {doc.page_content}" for doc, score in results]
            return "\n\n".join(context_parts)
        except Exception as e:
            logger.error(f"Retrieval error: {e}")
            return "Error retrieving information"
    
    def search_documents(self, query: str, k: int = 5) -> List[Dict[str, Any]]:
        """Search documents and return detailed results with metadata."""
        try:
            if not self.vectorstore:
                return []
            results = self.vectorstore.similarity_search_with_score(query, k=k)
            return [
                {
                    "content": doc.page_content[:500] + "..." if len(doc.page_content) > 500 else doc.page_content,
                    "metadata": doc.metadata,
                    "relevance": 1 - score,
                    "source": doc.metadata.get("source", "unknown")
                } for doc, score in results
            ]
        except Exception as e:
            logger.error(f"Search error: {e}")
            return []
    
    def get_collection_stats(self) -> Dict[str, Any]:
        """Get ChromaDB collection statistics."""
        try:
            if not self.chroma_client:
                return {"error": "ChromaDB not initialized"}
            collections = self.chroma_client.list_collections()
            stats = {
                "persist_directory": self.persist_directory,
                "collections": [col.name for col in collections],
                "current_collection": self.collection_name,
                "document_count": self.chroma_client.get_collection(self.collection_name).count()
                if self.collection_name in [col.name for col in collections] else 0
            }
            return stats
        except Exception as e:
            logger.error(f"Collection stats error: {e}")
            return {"error": str(e)}
    
    def reset_vectorstore(self) -> None:
        """Reset ChromaDB vectorstore."""
        try:
            if self.collection_name in [col.name for col in self.chroma_client.list_collections()]:
                self.chroma_client.delete_collection(self.collection_name)
                logger.info(f"Deleted collection: {self.collection_name}")
            self.vectorstore = None
            logger.info("Vectorstore reset")
        except Exception as e:
            logger.error(f"Reset error: {e}")

In [ ]:

            

class InsuranceMultiAgentSystem:
    """Multi-agent system for insurance queries with realistic workflow."""
    
    def __init__(self, rag_system: InsuranceRAGSystem):
        """Initialize the multi-agent system with RAG integration."""
        self.rag_system = rag_system
        self.agents = {}
        self.group_chat = None
        self.manager = None
        self.setup_agents()
        self.setup_group_chat()
    
    def query_classifier(self, query: str) -> str:
        """Classify query to determine the appropriate specialist."""
        query_lower = query.lower()
        if any(keyword in query_lower for keyword in ["claim", "accident", "file a claim"]):
            return "ClaimsSpecialist"
        elif any(keyword in query_lower for keyword in ["policy", "coverage", "premium"]):
            return "PolicyAdvisor"
        elif any(keyword in query_lower for keyword in ["compliance", "regulation", "law"]):
            return "ComplianceOfficer"
        elif any(keyword in query_lower for keyword in ["risk", "application", "underwriting"]):
            return "UnderwritingSpecialist"
        elif any(keyword in query_lower for keyword in ["fraud", "suspicious"]):
            return "FraudDetection"
        return "PolicyAdvisor"  # Default specialist
    
    def setup_agents(self) -> None:
        """Initialize specialized agents with AutoGen configuration."""
        config_list = [{
            "model": AZURE_CONFIG["gpt_deployment"],
            "api_type": "azure",
            "base_url": AZURE_CONFIG["base_url"],
            "api_key": AZURE_CONFIG["api_key"],
            "api_version": AZURE_CONFIG["api_version"]
        }]
        llm_config = {
            "config_list": config_list,
            "temperature": 0.1,
            "timeout": 60,
            "cache_seed": 42
        }
        
        # Customer Service Agent
        self.agents["customer_service"] = ConversableAgent(
            name="CustomerService",
            system_message="""You are the primary customer interface. Responsibilities:
            - Classify queries and delegate to the appropriate specialist:
              - Claims (e.g., "claim", "accident"): ClaimsSpecialist
              - Policy (e.g., "policy", "coverage"): PolicyAdvisor
              - Compliance (e.g., "compliance", "law"): ComplianceOfficer
              - Underwriting (e.g., "risk", "application"): UnderwritingSpecialist
              - Fraud (e.g., "fraud", "suspicious"): FraudDetection
              - Default: PolicyAdvisor
            - Delegate with: "This query should be handled by [AgentName]: [query]"
            - Wait for specialist response, then present it: "Response from [AgentName]: [answer] TERMINATE"
            - Maintain conversation history and ensure customer satisfaction.""",
            llm_config=llm_config,
            human_input_mode="NEVER",
            max_consecutive_auto_reply=3,
            code_execution_config={"use_docker": False}
        )
        
        # Knowledge Retriever Agent
        self.agents["retriever"] = RetrieveUserProxyAgent(
            name="KnowledgeRetriever",
            system_message="""You are the Knowledge Retriever. Responsibilities:
            - Respond only when consulted by a specialist.
            - Search ChromaDB for relevant policy information.
            - Provide accurate citations with document sources.
            - State if information is unavailable.""",
            human_input_mode="NEVER",
            max_consecutive_auto_reply=3,
            retrieve_config={
                "task": "qa",
                "docs_path": "./data",
                "chunk_token_size": 1500,
                "model": config_list[0]["model"],
                "collection_name": self.rag_system.collection_name,
                "get_or_create": True
            },
            code_execution_config={"use_docker": False}
        )
        
        # Claims Specialist Agent
        self.agents["claims_agent"] = ConversableAgent(
            name="ClaimsSpecialist",
            system_message="""You are the Claims Specialist. Responsibilities:
            - Respond only when delegated by CustomerService or consulted.
            - Guide customers through claims processes.
            - Consult KnowledgeRetriever for policy details if needed, e.g., "KnowledgeRetriever, provide claims process details."
            - Request ComplianceOfficer review for regulatory compliance.
            - Return response to CustomerService.""",
            llm_config=llm_config,
            human_input_mode="NEVER",
            max_consecutive_auto_reply=3,
            code_execution_config={"use_docker": False}
        )
        
        # Policy Advisor Agent
        self.agents["policy_advisor"] = ConversableAgent(
            name="PolicyAdvisor",
            system_message="""You are the Policy Advisor. Responsibilities:
            - Respond only when delegated by CustomerService or consulted.
            - Handle policy-related queries (e.g., coverage, premiums).
            - Consult KnowledgeRetriever for policy documents if needed.
            - Return response to CustomerService.""",
            llm_config=llm_config,
            human_input_mode="NEVER",
            max_consecutive_auto_reply=3,
            code_execution_config={"use_docker": False}
        )
        
        # Compliance Officer Agent
        self.agents["compliance_agent"] = ConversableAgent(
            name="ComplianceOfficer",
            system_message="""You are the Compliance Officer. Responsibilities:
            - Respond only when consulted by a specialist.
            - Verify regulatory compliance of responses.
            - Ensure state-specific regulations are followed.
            - Provide compliance insights or corrections.""",
            llm_config=llm_config,
            human_input_mode="NEVER",
            max_consecutive_auto_reply=2,
            code_execution_config={"use_docker": False}
        )
        
        # Underwriting Specialist Agent
        self.agents["underwriting_agent"] = ConversableAgent(
            name="UnderwritingSpecialist",
            system_message="""You are the Underwriting Specialist. Responsibilities:
            - Respond only when delegated by CustomerService or consulted.
            - Handle underwriting-related queries (e.g., risk assessment).
            - Consult KnowledgeRetriever for guidelines if needed.""",
            llm_config=llm_config,
            human_input_mode="NEVER",
            max_consecutive_auto_reply=3,
            code_execution_config={"use_docker": False}
        )
        
        # Fraud Detection Agent
        self.agents["fraud_detector"] = ConversableAgent(
            name="FraudDetection",
            system_message="""You are the Fraud Detection Analyst. Responsibilities:
            - Respond only when delegated by CustomerService or consulted.
            - Analyze queries for fraud indicators.
            - Consult KnowledgeRetriever for verification if needed.""",
            llm_config=llm_config,
            human_input_mode="NEVER",
            max_consecutive_auto_reply=2,
            code_execution_config={"use_docker": False}
        )
        
        # Supervisor Agent
        self.agents["supervisor"] = ConversableAgent(
            name="Supervisor",
            system_message="""You are the Supervisor. Responsibilities:
            - Respond only if escalated or for final approval.
            - Finalize responses after specialist and ComplianceOfficer input.
            - Terminate with 'TERMINATE' when complete.""",
            llm_config=llm_config,
            human_input_mode="NEVER",
            max_consecutive_auto_reply=2,
            code_execution_config={"use_docker": False},
            is_termination_msg=lambda msg: "TERMINATE" in msg.get("content", "").upper()
        )
    
    def setup_group_chat(self) -> None:
        """Setup group chat with all agents for collaboration."""
        agents_list = [
            self.agents["customer_service"],
            self.agents["retriever"],
            self.agents["claims_agent"],
            self.agents["policy_advisor"],
            self.agents["compliance_agent"],
            self.agents["underwriting_agent"],
            self.agents["fraud_detector"],
            self.agents["supervisor"]
        ]
        self.group_chat = GroupChat(
            agents=agents_list,
            messages=[],
            max_round=12,
            speaker_selection_method="auto",
            allow_repeat_speaker=False
        )
        self.manager = GroupChatManager(
            groupchat=self.group_chat,
            llm_config={
                "config_list": [{
                    "model": AZURE_CONFIG["gpt_deployment"],
                    "api_type": "azure",
                    "base_url": AZURE_CONFIG["base_url"],
                    "api_key": AZURE_CONFIG["api_key"],
                    "api_version": AZURE_CONFIG["api_version"]
                }],
                "temperature": 0.1
            },
            code_execution_config={"use_docker": False}
        )
    
    def process_query(self, query: str) -> Dict[str, Any]:
        """Process customer query through multi-agent system with real-time logging."""
        try:
            logger.info(f"Processing query: {query}")
            context = self.rag_system.retrieve_context(query)
            primary_agent = self.query_classifier(query)
            logger.info(f"Primary agent assigned: {primary_agent}")
            
            enhanced_query = f"""
            ## INSURANCE QUERY PROCESSING ##
            Customer Query: {query}
            Primary Agent: {primary_agent}
            Relevant Knowledge Base Context:
            {context}
            """
            
            chat_result = self.agents["customer_service"].initiate_chat(
                self.manager,
                message=enhanced_query,
                clear_history=True
            )
            
            final_response = ""
            agents_involved = []
            if chat_result and chat_result.chat_history:
                for msg in chat_result.chat_history:
                    if "name" in msg:
                        agents_involved.append(msg["name"])
                        logger.info(f"Agent {msg['name']} triggered")
                    if msg.get("name") == "CustomerService" and "TERMINATE" in msg.get("content", "").upper():
                        final_response = msg["content"].split("TERMINATE")[0].strip()
                        break
                if not final_response:
                    final_response = chat_result.chat_history[-1].get("content", "")
            
            return {
                "query": query,
                "response": final_response,
                "primary_agent": primary_agent,
                "agents_involved": list(set(agents_involved)),
                "context_used": bool(context.strip())
            }
        except Exception as e:
            logger.error(f"Processing error: {e}")
            return {
                "query": query,
                "response": f"I apologize, but I encountered an error: {str(e)}",
                "primary_agent": "Unknown",
                "agents_involved": [],
                "context_used": False
            }

    def simple_query(self, query: str) -> str:
        """Handle simple queries with direct RAG for quick responses."""
        try:
            logger.info(f"Processing simple query: {query}")
            context = self.rag_system.retrieve_context(query)
            prompt = f"""
            [Insurance Expert Mode]
            Answer concisely using ONLY the provided context:
            Query: {query}
            Context:
            {context}
            Rules:
            1. Answer in 1-2 paragraphs max
            2. Cite specific policy details when available
            3. If context is insufficient, state: "Based on my knowledge base: [general answer]"
            4. Never speculate beyond provided context
            """
            response = self.rag_system.llm.invoke([HumanMessage(content=prompt)])
            logger.info("Simple query processed successfully")
            return response.content
        except Exception as e:
            logger.error(f"Simple query error: {e}")
            return f"I apologize, but I encountered an error: {str(e)}"

In [ ]:


def main() -> None:
    """Main function to run the insurance multi-agent RAG system."""
    print("\n" + "="*60)
    print("INSURANCE MULTI-AGENT RAG SYSTEM WITH CHROMADB")
    print("="*60)
    
    try:
        logger.info("Initializing RAG system")
        rag_system = InsuranceRAGSystem(
            data_folder="data",
            persist_directory="chroma_db"
        )
        
        stats = rag_system.get_collection_stats()
        if stats.get("document_count", 0) == 0:
            logger.info("Loading documents")
            documents = rag_system.load_and_process_documents()
            rag_system.create_or_load_vectorstore(documents)
        else:
            rag_system.create_or_load_vectorstore()
        
        logger.info("Setting up multi-agent system")
        agent_system = InsuranceMultiAgentSystem(rag_system)
        
        print("\n System initialized successfully!")
        print(f" Agents: {', '.join(agent_system.agents.keys())}")
        stats = rag_system.get_collection_stats()
        print(f"\n ChromaDB: {stats['current_collection']} ({stats.get('document_count', 0)} docs)")
        
        print("\n" + "="*60)
        print("INTERACTIVE MODE - Ask insurance questions")
        print("Commands: stats, search [query], reload, reset, quit")
        print("="*60)
        
        while True:
            user_input = input("\n Your question: ").strip()
            
            if user_input.lower() in ['quit', 'exit']:
                print("\n Thank you for using the Insurance Assistant!")
                break
                
            if not user_input:
                continue
                
            if user_input.lower() == 'stats':
                stats = rag_system.get_collection_stats()
                print("\n ChromaDB Stats:")
                for key, value in stats.items():
                    print(f"  {key}: {value}")
                continue
                
            if user_input.lower().startswith('search '):
                query = user_input[7:].strip()
                print(f"\n Searching for: '{query}'")
                results = rag_system.search_documents(query)
                if results:
                    for i, res in enumerate(results, 1):
                        print(f"\n{i}. Source: {res['source']} (Relevance: {res['relevance']:.2f})")
                        print(f"   Content: {res['content']}")
                else:
                    print("No results found")
                continue
                
            if user_input.lower() == 'reload':
                confirm = input("Reload documents? (y/n): ")
                if confirm.lower() == 'y':
                    documents = rag_system.load_and_process_documents()
                    if documents:
                        rag_system.create_or_load_vectorstore(documents, force_recreate=True)
                        print(" Documents reloaded")
                continue
                
            if user_input.lower() == 'reset':
                confirm = input("Reset ChromaDB? All data will be deleted. (y/n): ")
                if confirm.lower() == 'y':
                    rag_system.reset_vectorstore()
                    print(" ChromaDB reset")
                continue
                
            print("\n Processing your query...")
            try:
                result = agent_system.process_query(user_input)
                print("\n Response:")
                print("-" * 60)
                print(result["response"])
                print("-" * 60)
                print(f" Primarily handled by: {result['primary_agent']}")
                print(f" Agents involved: {', '.join(result['agents_involved'])}")
                if result["context_used"]:
                    print(" Knowledge base was consulted")
                results = rag_system.search_documents(user_input, k=2)
                if results:
                    print("\n Related Information Sources:")
                    for i, res in enumerate(results, 1):
                        print(f"{i}. {res['source']} (Relevance: {res['relevance']:.2f})")
            except Exception as e:
                logger.error(f"Query processing error: {e}")
                print(f" Error: {e}")
                
    except KeyboardInterrupt:
        print("\n Goodbye!")
    except Exception as e:
        logger.error(f"Fatal error: {e}")
        import traceback
        traceback.print_exc()

In [ ]:


if __name__ == "__main__":
    main()

2025-06-12 20:26:07,356 - __main__ - INFO - Initializing RAG system



INSURANCE MULTI-AGENT RAG SYSTEM WITH CHROMADB


2025-06-12 20:26:08,590 - __main__ - INFO - Azure OpenAI clients initialized successfully
2025-06-12 20:26:08,609 - __main__ - INFO - ChromaDB client initialized at: chroma_db
2025-06-12 20:26:08,613 - __main__ - INFO - Loading existing collection: insurance_documents
2025-06-12 20:26:08,629 - __main__ - INFO - Loaded 316 documents
2025-06-12 20:26:08,633 - __main__ - INFO - Setting up multi-agent system
2025-06-12 20:26:37,398 - tensorflow - WARNING - From c:\Users\dhanu\miniconda3\envs\idk\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

2025-06-12 20:26:38,487 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-06-12 20:26:42,171 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.



✅ System initialized successfully!
🤖 Agents: customer_service, retriever, claims_agent, policy_advisor, compliance_agent, underwriting_agent, fraud_detector, supervisor

🔵 ChromaDB: insurance_documents (316 docs)

INTERACTIVE MODE - Ask insurance questions
Commands: stats, search [query], reload, reset, quit


2025-06-12 20:27:09,720 - __main__ - INFO - Processing query: suggest me some better HSLA policies?


...


Next speaker: Supervisor


>>>>>>>> TERMINATING RUN (8c873ec6-d978-417d-a3be-b07d2dabb245): Termination message condition on agent 'Supervisor' met

>>>>>>>> TERMINATING RUN (315fe0b3-073b-4736-a28e-9462d1485b3d): No reply generated


2025-06-12 20:27:26,176 - __main__ - INFO - Agent CustomerService triggered
2025-06-12 20:27:26,183 - __main__ - INFO - Agent PolicyAdvisor triggered
2025-06-12 20:27:26,184 - __main__ - INFO - Agent CustomerService triggered



💡 Response:
------------------------------------------------------------
Response from PolicyAdvisor: Based on your query, here are some HLA policies that might suit your needs:

1. **HLA Life Essential**:
   - Affordable premiums starting as low as RM 20 per month.
   - Yearly renewable basic plan offering essential coverage.
   - Flexibility to enhance coverage for medical, critical illness, accidents, total & permanent disability (TPD), and old age disablement (OAD).
   - Suitable for individuals seeking a strong foundation of protection without heavy financial commitment.

2. **HLA Asset Elite**:
   - Designed to safeguard your family’s financial wellbeing in case of unfortunate incidents.
   - Provides comprehensive protection to ensure your family’s stability even when you’re no longer around.
   - Ideal for individuals prioritizing family security and financial planning.

3. **HLA Prime Secure**:
   - Investment-linked plan with limited premium payment terms.
   - Offers a guar

2025-06-12 20:27:27,076 - httpx - INFO - HTTP Request: POST https://idkrag.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2025-01-01-preview "HTTP/1.1 200 OK"



🔍 Related Information Sources:
1. data\Data\Hong Leong Assurance.pdf (Relevance: 0.60)
2. data\Data\HLA Asset Elite Product Brochure.pdf (Relevance: 0.58)

👋 Thank you for using the Insurance Assistant!
